In [1]:
import requests
#!conda install -c conda-forge wikipedia
from time import sleep
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

In [2]:

def rowgetDataText(tr, coltag='td'): # td (data) or th (header)
    cols = []
    for td in tr.find_all(coltag):
        cols.append(td.get_text(strip=True))
    return cols

def tableDataText(table):       
    rows = []
    trs = table.find_all('tr')
    headerow = rowgetDataText(trs[0], 'th')
    if headerow: # if there is a header row include first
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs: # for every table row
        rows.append(rowgetDataText(tr, 'td')) # data row
    return rows

table = soup.find('table', {'class': 'wikitable sortable'})
list_table = tableDataText(table)
list_table

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Harbourfront'],
 ['M6A', 'North York', 'Lawrence Heights'],
 ['M6A', 'North York', 'Lawrence Manor'],
 ['M7A', "Queen's Park", 'Not assigned'],
 ['M8A', 'Not assigned', 'Not assigned'],
 ['M9A', "Queen's Park", "Queen's Park"],
 ['M1B', 'Scarborough', 'Rouge'],
 ['M1B', 'Scarborough', 'Malvern'],
 ['M2B', 'Not assigned', 'Not assigned'],
 ['M3B', 'North York', 'Don Mills North'],
 ['M4B', 'East York', 'Woodbine Gardens'],
 ['M4B', 'East York', 'Parkview Hill'],
 ['M5B', 'Downtown Toronto', 'Ryerson'],
 ['M5B', 'Downtown Toronto', 'Garden District'],
 ['M6B', 'North York', 'Glencairn'],
 ['M7B', 'Not assigned', 'Not assigned'],
 ['M8B', 'Not assigned', 'Not assigned'],
 ['M9B', 'Etobicoke', 'Cloverdale'],
 ['M9B', 'Etobicoke', 'Islington'],
 ['M9B', 'Et

In [3]:
import pandas as pd

dftable = pd.DataFrame(list_table[1:], columns=list_table[0])
dftable.head(4)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village


Drop not assigned borough

In [4]:
dftable = dftable[dftable.Borough != 'Not assigned']
dftable.shape

(210, 3)

  Merege multiple rows into one

In [5]:
#df = dftable.groupby(['Postcode'])['Neighbourhood'].apply(' ,'.join).reset_index()
df = dftable.groupby(['Postcode', 'Borough'])['Neighbourhood'].agg(' ,'.join).reset_index()
df.head(4)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge ,Malvern"
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,Scarborough,Woburn


Replace Not assigned in Neighbourhood

In [6]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.head(4)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge ,Malvern"
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union"
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill"
3,M1G,Scarborough,Woburn


Check dimension

In [7]:
df.shape

(103, 3)

In [8]:
#df_can = pd.read_csv('C:/Users/xzhang/Downloads/Geospatial_Coordinates.csv')#, index_col = 0)
df_can = pd.read_csv('http://cocl.us/Geospatial_data')#, index_col = 0)

merge two datasets

In [9]:
dfm = df.merge(df_can, left_on='Postcode', right_on='Postal Code').drop(columns = 'Postal Code')
dfm.head(4)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge ,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917


get the location

In [10]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [11]:
# create map of New York using latitude and longitude values
#!conda install -c conda-forge folium=0.5.0
neighborhoods = dfm
import folium # map rendering library
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Get data using foursqaure API

In [68]:
CLIENT_ID = 'BMWVCTTW5K0L51TMQEHKDRS0ZLHCVOJEIXIW4FOUZUMPSNXP' # your Foursquare ID
CLIENT_SECRET = '2XTUVEL1DCZKVLXC0CCGT40WWQ3FNXPTKXRGBXOFSZYNPC2K' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BMWVCTTW5K0L51TMQEHKDRS0ZLHCVOJEIXIW4FOUZUMPSNXP
CLIENT_SECRET:2XTUVEL1DCZKVLXC0CCGT40WWQ3FNXPTKXRGBXOFSZYNPC2K


In [13]:
dfm.loc[0, 'Neighbourhood']
neighborhood_latitude = dfm.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = dfm.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = dfm.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))



Latitude and longitude values of Rouge ,Malvern are 43.806686299999996, -79.19435340000001.


In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=BMWVCTTW5K0L51TMQEHKDRS0ZLHCVOJEIXIW4FOUZUMPSNXP&client_secret=2XTUVEL1DCZKVLXC0CCGT40WWQ3FNXPTKXRGBXOFSZYNPC2K&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dced840949393001b070f4a'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
dfm_venues = getNearbyVenues(names=dfm['Neighbourhood'],
                                   latitudes=dfm['Latitude'],
                                   longitudes=dfm['Longitude']
                                  )



Rouge ,Malvern
Highland Creek ,Rouge Hill ,Port Union
Guildwood ,Morningside ,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park ,Ionview ,Kennedy Park
Clairlea ,Golden Mile ,Oakridge
Cliffcrest ,Cliffside ,Scarborough Village West
Birch Cliff ,Cliffside West
Dorset Park ,Scarborough Town Centre ,Wexford Heights
Maryvale ,Wexford
Agincourt
Clarks Corners ,Sullivan ,Tam O'Shanter
Agincourt North ,L'Amoreaux East ,Milliken ,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview ,Henry Farm ,Oriole
Bayview Village
Silver Hills ,York Mills
Newtonbrook ,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park ,Don Mills South
Bathurst Manor ,Downsview North ,Wilson Heights
Northwood Park ,York University
CFB Toronto ,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens ,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West ,

In [20]:
print(dfm_venues.shape)
dfm_venues.head()

(2250, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge ,Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


Clean venues data and prepare for analysis

In [21]:
dfm_venues.groupby('Neighborhood').count()
print('There are {} uniques categories.'.format(len(dfm_venues['Venue Category'].unique())))

There are 266 uniques categories.


In [22]:
# one hot encoding
dfm_onehot = pd.get_dummies(dfm_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dfm_onehot['Neighborhood'] = dfm_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dfm_onehot.columns[-1]] + list(dfm_onehot.columns[:-1])
dfm_onehot = dfm_onehot[fixed_columns]

dfm_onehot.head()

dfm_onehot.shape

(2250, 266)

In [23]:
dfm_grouped = dfm_onehot.groupby('Neighborhood').mean().reset_index()
dfm_grouped.shape


(99, 266)

In [24]:
num_top_venues = 5

for hood in dfm_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = dfm_grouped[dfm_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide ,King ,Richmond----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2                  Bar  0.04
3           Steakhouse  0.04
4  American Restaurant  0.03


----Agincourt----
                       venue  freq
0                     Lounge   0.2
1               Skating Rink   0.2
2  Latin American Restaurant   0.2
3             Breakfast Spot   0.2
4             Clothing Store   0.2


----Agincourt North ,L'Amoreaux East ,Milliken ,Steeles East----
                             venue  freq
0                       Playground   0.5
1                             Park   0.5
2         Mediterranean Restaurant   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Albion Gardens ,Beaumond Heights ,Humbergate ,Jamestown ,Mount Olive ,Silverstone ,South Steeles ,Thistletown----
            venue  freq
0     Pizza Place  0.12
1     Video Store  0.12
2        Pharmacy  0.12
3      Beer Store  0.12
4  Sandwic

In [25]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [26]:
import numpy as np # library to handle data in a vectorized manner
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = dfm_grouped['Neighborhood']

for ind in np.arange(dfm_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dfm_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide ,King ,Richmond",Coffee Shop,Café,Bar,Steakhouse,Breakfast Spot,Thai Restaurant,Restaurant,Bakery,Hotel,Sushi Restaurant
1,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Clothing Store,Skating Rink,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop,Empanada Restaurant
2,"Agincourt North ,L'Amoreaux East ,Milliken ,St...",Playground,Park,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
3,"Albion Gardens ,Beaumond Heights ,Humbergate ,...",Pharmacy,Grocery Store,Pizza Place,Fried Chicken Joint,Video Store,Beer Store,Sandwich Place,Fast Food Restaurant,Discount Store,Dessert Shop
4,"Alderwood ,Long Branch",Pizza Place,Gym,Skating Rink,Dance Studio,Coffee Shop,Pub,Sandwich Place,Pharmacy,Airport Service,Deli / Bodega


In [27]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [36]:
# set number of clusters
kclusters = 5

dfm_grouped_clustering = dfm_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dfm_grouped_clustering)

# check cluster labels generated for each row in the dataframe
len(kmeans.labels_)#[0:10] 
kmeans.labels_

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 4, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 4, 0, 0, 0,
       4, 0, 0, 0, 0, 4, 0, 4, 0, 3, 0, 0, 0, 2, 0, 0, 0, 0, 4, 0, 0, 0,
       4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1], dtype=int32)

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted


,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Adelaide ,King ,Richmond",Coffee Shop,Café,Bar,Steakhouse,Breakfast Spot,Thai Restaurant,Restaurant,Bakery,Hotel,Sushi Restaurant
1,0,Agincourt,Latin American Restaurant,Lounge,Breakfast Spot,Clothing Store,Skating Rink,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop,Empanada Restaurant
2,4,"Agincourt North ,L'Amoreaux East ,Milliken ,St...",Playground,Park,Women's Store,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
3,0,"Albion Gardens ,Beaumond Heights ,Humbergate ,...",Pharmacy,Grocery Store,Pizza Place,Fried Chicken Joint,Video Store,Beer Store,Sandwich Place,Fast Food Restaurant,Discount Store,Dessert Shop
4,0,"Alderwood ,Long Branch",Pizza Place,Gym,Skating Rink,Dance Studio,Coffee Shop,Pub,Sandwich Place,Pharmacy,Airport Service,Deli / Bodega
5,0,"Bathurst Manor ,Downsview North ,Wilson Heights",Coffee Shop,Sushi Restaurant,Pharmacy,Pizza Place,Chinese Restaurant,Middle Eastern Restaurant,Restaurant,Deli / Bodega,Sandwich Place,Bridal Shop
6,0,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
7,0,"Bedford Park ,Lawrence Manor East",Italian Restaurant,Coffee Shop,Pub,Pizza Place,Butcher,Sushi Restaurant,Café,Indian Restaurant,Fast Food Restaurant,Sandwich Place
8,0,Berczy Park,Coffee Shop,Bakery,Steakhouse,Beer Bar,Cocktail Bar,Cheese Shop,Farmers Market,Café,Seafood Restaurant,Clothing Store
9,0,"Birch Cliff ,Cliffside West",Skating Rink,College Stadium,Café,General Entertainment,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


In [63]:
manhattan_merged = dfm

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

manhattan_merged#.head() # check the last columns!
new = manhattan_merged.dropna()
new.dtypes
#
new['Cluster Labels'] = new['Cluster Labels'].astype(int)
new

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge ,Malvern",43.806686,-79.194353,3,Fast Food Restaurant,Women's Store,Deli / Bodega,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant
1,M1C,Scarborough,"Highland Creek ,Rouge Hill ,Port Union",43.784535,-79.160497,0,Bar,Women's Store,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
2,M1E,Scarborough,"Guildwood ,Morningside ,West Hill",43.763573,-79.188711,0,Mexican Restaurant,Spa,Pizza Place,Electronics Store,Rental Car Location,Breakfast Spot,Medical Center,Intersection,Diner,Dessert Shop
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Pharmacy,Korean Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0,Fried Chicken Joint,Bakery,Hakka Restaurant,Athletics & Sports,Bank,Thai Restaurant,Caribbean Restaurant,Women's Store,Discount Store,Dim Sum Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,0,Playground,Cosmetics Shop,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
6,M1K,Scarborough,"East Birchmount Park ,Ionview ,Kennedy Park",43.727929,-79.262029,0,Coffee Shop,Discount Store,Department Store,Convenience Store,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant
7,M1L,Scarborough,"Clairlea ,Golden Mile ,Oakridge",43.711112,-79.284577,0,Bakery,Bus Line,Park,Bus Station,Fast Food Restaurant,Intersection,Soccer Field,Construction & Landscaping,Comfort Food Restaurant,Empanada Restaurant
8,M1M,Scarborough,"Cliffcrest ,Cliffside ,Scarborough Village West",43.716316,-79.239476,0,Motel,American Restaurant,Women's Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
9,M1N,Scarborough,"Birch Cliff ,Cliffside West",43.692657,-79.264848,0,Skating Rink,College Stadium,Café,General Entertainment,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant


In [64]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


In [65]:

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


In [66]:

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(new['Latitude'], new['Longitude'], new['Neighbourhood'], new['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Clustering check

In [ ]:
new.loc[new['Cluster Labels'] == 0, new.columns[[1] + list(range(5, new.shape[1]))]]

In [67]:
new.loc[new['Cluster Labels'] == 1, new.columns[[1] + list(range(5, new.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0,Bar,Women's Store,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop
2,Scarborough,0,Mexican Restaurant,Spa,Pizza Place,Electronics Store,Rental Car Location,Breakfast Spot,Medical Center,Intersection,Diner,Dessert Shop
3,Scarborough,0,Coffee Shop,Pharmacy,Korean Restaurant,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore
4,Scarborough,0,Fried Chicken Joint,Bakery,Hakka Restaurant,Athletics & Sports,Bank,Thai Restaurant,Caribbean Restaurant,Women's Store,Discount Store,Dim Sum Restaurant
5,Scarborough,0,Playground,Cosmetics Shop,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
6,Scarborough,0,Coffee Shop,Discount Store,Department Store,Convenience Store,Women's Store,Donut Shop,Dim Sum Restaurant,Diner,Dog Run,Doner Restaurant
7,Scarborough,0,Bakery,Bus Line,Park,Bus Station,Fast Food Restaurant,Intersection,Soccer Field,Construction & Landscaping,Comfort Food Restaurant,Empanada Restaurant
8,Scarborough,0,Motel,American Restaurant,Women's Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
9,Scarborough,0,Skating Rink,College Stadium,Café,General Entertainment,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
10,Scarborough,0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Furniture / Home Store,Chinese Restaurant,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner


In [ ]:
new.loc[new['Cluster Labels'] == 2, new.columns[[1] + list(range(5, new.shape[1]))]]

In [ ]:
new.loc[new['Cluster Labels'] == 3, new.columns[[1] + list(range(5, new.shape[1]))]]

In [ ]:
new.loc[new['Cluster Labels'] == 4, new.columns[[1] + list(range(5, new.shape[1]))]]